In [1]:
#arcpy to process LIDAR data    
%load_ext autoreload
%autoreload 2

from arcpy import env
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcpy.sa import *

import sys
sys.path.append("..")

from src.features.create_rasters import *
from src.features.arcpy_scripts import *
from src.features.custom_functions import * 


In [ ]:
#define variables - not sure if this is necessary? probably not?

#project geodatabase
#env.workspace = r'C:\Users\rbowers\Desktop\cool_roof_analysis_offnetwork.gdb'
#env.overwriteOutput = True

#dataset file path
#las_dataset = r'I:\Imagery\MassGIS_LAS_files\MMC_lasdataset.lasd'

#arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1983 StatePlane Massachusetts FIPS 2001 (Meters)")


### Run these only if need be 

- las_dataset was last created for all MMC on 5/1/2024
- heat_score_mmc was last created for all MMC on 5/1/2024

In [ ]:
#only run this if need be [last run on all MMC: 5/1/2024]
from src.data.make_dataset import heat_fp, las_folder


#las_dataset = create_las_dataset(las_folder=las_folder, las_dataset=las_dataset)

heat_score_mmc = get_heat_score_mmc(heat_index_fp=heat_fp)

### First create slope and intensity rasters for each municipality

Result in geodatabase should be just a slope and intensity raster for each muni - note, as of 5/22, this can be skipped (unless you need to rerun any of them)

In [ ]:
mmc_town_names = ['Arlington', 'Boston', 'Braintree', 'Brookline', 'Cambridge', 'Chelsea', 
             'Everett', 'Malden', 'Medford', 'Melrose', 'Newton', 'Quincy', 'Revere', 
             'Somerville', 'Watertown', 'Winthrop']


#note that Revere was removed from this list since it was the test one 

from datetime import datetime

for town_name in mmc_town_names:
    
    print(town_name + ' processing starting at ' + str(datetime.now()))


    las_dataset = create_las_dataset(town_name = town_name, 
                                     las_folder=las_folder)
    

    ndsm_raster = create_ndsm_raster(town_name=town_name,
                                    las_dataset=las_dataset)
    

    slope_raster = create_slope_raster(town_name=town_name,
                                        ndsm_raster=ndsm_raster)
    
    #removing aspect for now, may include later to refine the flat roof analysis
    #aspect_raster = create_aspect_raster(town_name=town_name,
    #                                     ndsm_raster=ndsm_raster)


    intensity_raster = create_intensity_raster(town_name=town_name, 
                                                las_dataset=las_dataset)


### Then create enriched roofprint polygon datasets

In [2]:
from src.data.make_dataset import mmc_heat_blocks
from datetime import datetime


mmc_town_names = ['Arlington', 
                  #'Boston', 
                  'Braintree', 'Brookline', 'Cambridge', 'Chelsea', 
             'Everett', 'Malden', 'Medford', 'Melrose', 'Newton', 'Quincy', 'Revere', 
             'Somerville', 'Watertown', 'Winthrop']

for town_name in mmc_town_names:
  
    print(town_name + ' processing starting at ' + str(datetime.now()))

    cool_roof_footprints_layer = os.path.join(cool_roofs_gdb, (town_name + '_footprints'))
    
    if arcpy.Exists(cool_roof_footprints_layer): #an existing out_slope_raster would mean this town has already been processed
        pass


    else:
    
        rooftops_layer = make_coolroof_roofprints_layer(town_name=town_name)
        
        #input for the cool_roof_process() function has to be a gdf
        rooftops_gdf = gpd.read_file(cool_roofs_gdb, layer=(town_name + '_footprints'))

        town_structures_w_info = cool_roof_process(town_name=town_name, 
                                                    rooftops_layer=rooftops_gdf,
                                                    heat_blocks_layer=mmc_heat_blocks)


Arlington processing starting at 2024-05-22 12:48:57.386050
Braintree processing starting at 2024-05-22 12:49:09.361271
Brookline processing starting at 2024-05-22 12:49:09.645385
Cambridge processing starting at 2024-05-22 12:49:09.988957
Chelsea processing starting at 2024-05-22 12:49:10.273353
Everett processing starting at 2024-05-22 12:49:10.538341
Malden processing starting at 2024-05-22 12:49:10.819359
Medford processing starting at 2024-05-22 12:49:11.099107
Melrose processing starting at 2024-05-22 12:49:11.390372
Newton processing starting at 2024-05-22 12:49:11.659966
Quincy processing starting at 2024-05-22 12:49:11.943078
Revere processing starting at 2024-05-22 12:49:12.246517
Somerville processing starting at 2024-05-22 12:49:12.530084
Watertown processing starting at 2024-05-22 12:49:12.800270
Winthrop processing starting at 2024-05-22 12:49:13.123215
get town boundary
town footprints
extract by mask
reclassify
joining
intensity
extract by mask, intensity
reclassify, in

c:\Users\rbowers\.conda\envs\ArcPy\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\rbowers\scripting-projects\clean-energy-mmc-cool-roofs\notebooks\..\src\features\custom_functions.py:695: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  muni_structures_join_parcels.to_file(export)


In [ ]:
## CREATE LAS DATASET ##

point_selection_method = 'CLOSEST_TO_CENTER'

arcpy.CreateLasDataset_management(input=las_folder,
                                  out_las_dataset=las_dataset,
                                  compute_stats='COMPUTE_STATS')
                                  